# `ABOUT THE DATASET`

1) id:unique id for a news article

2) title: the title of a news article

3) author: author of the news article

4) text: the text of the article, could be incomplete

5) label: a label that marks whether the news article is real or fake.


1 : fake news

0 : real news

# `IMPORTING LIBRARIES`

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
# import matplotlib as plt
# plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# `LOADING THE DATASET`

In [ ]:
# LOADING THE DATASET INTO PANDAS DATAFRAME.
new_dataset = pd.read_csv("/content/train.csv.zip")
new_dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


# `EXPLORATORY DATA ANALYSIS (EDA)`

In [ ]:
sns.pairplot(new_dataset, hue='label')

In [ ]:
corelation = new_dataset.corr()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(corelation , annot=True , linewidths=0.5 , cmap="YlGnBu")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


# `DATA PREPROCESSING`

In [ ]:
new_dataset.shape

(20800, 5)

In [ ]:
# PRINT THE FIRST 5 ROWS 
new_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
new_dataset.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [ ]:
# COUNTING THE NUMBER OF MISSING VALUES IN EACH COLUMNS.
new_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# REPLACING THE  NULL VALUES WITH EMPTY STRING.
new_dataset = new_dataset.fillna('')

WE ARE GOING TO COMBINE TITLE AND AUTHER TOGETHER TO CARRY OUT OUR PREDICTIONS 

In [ ]:
#MERGING THE AUTHOR AND TITLE COLUMN.
new_dataset['content'] = new_dataset['author'] + new_dataset['title']
print(new_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
# SEPARATING THE DATA AND LABEL.
x = new_dataset.drop(columns = 'label' , axis = 1)
print(x)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
y = new_dataset['label']
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


 STEMMING : STEMMING IS THE PROCEDURE OF REDUCING A PARTICULAR WORD TO ITS ROOT WORD.
 Example -> swimming , swim , swimmer. -> swim

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ' , content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [ ]:
new_dataset['content'] = new_dataset['content'].apply(stemming)

In [ ]:
print(new_dataset['content'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
# SEPARATING THE DATA AND LABEL.

X = new_dataset['content'].values
Y = new_dataset['label'].values


In [ ]:
print(X)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(20800,)

In [ ]:
# converting texual data to numerical data.
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 26340)	0.28088379401596425
  (0, 22724)	0.2552336018069161
  (0, 15019)	0.43006226759639316
  (0, 14555)	0.29177259684200296
  (0, 12782)	0.24619727512767195
  (0, 8022)	0.23133661742488731
  (0, 6273)	0.2839932825877813
  (0, 5969)	0.35488202138141456
  (0, 5006)	0.2472595823572816
  (0, 4211)	0.3625320323150658
  (0, 578)	0.2694167078545385
  (1, 27923)	0.36911845953845024
  (1, 11313)	0.24166773097712638
  (1, 8772)	0.5258635625386451
  (1, 5916)	0.31810058109638056
  (1, 4767)	0.23338756776626793
  (1, 3859)	0.45980466668763476
  (1, 3281)	0.18652439327549428
  (1, 2622)	0.3562953366945267
  (2, 26235)	0.3665032495181434
  (2, 16361)	0.43295215406038445
  (2, 9454)	0.30743020569262086
  (2, 8567)	0.3411947414020896
  (2, 5240)	0.40440534260277944
  (2, 5121)	0.5511414848555652
  :	:
  (20797, 25776)	0.08220218573989037
  (20797, 25319)	0.3119640221826561
  (20797, 22086)	0.24902354987792552
  (20797, 20778)	0.2729578683228216
  (20797, 20493)	0.249994989010826
  (20797, 17505

# `SPLITTING THE DATASET INTO TRAINING AND TEST DATA`

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,stratify=Y ,random_state=2)

# `CREATING AND TRAINING THE MODEL`

1)Logistic Regression : 

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

In [ ]:
# ACCURACY SCORE ON TRAINING DATA

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)


In [ ]:
print("Accuracy score of the training data :" , training_data_accuracy)

Accuracy score of the training data : 0.9719951923076923


In [ ]:
# ACCURACY SCORE ON TEST DATA

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)



In [ ]:
print("Accuracy score of the test data :" , test_data_accuracy)

Accuracy score of the test data : 0.9548076923076924


In [ ]:
# Confusion Matrix
predictions=model.predict(X_test)
cm=confusion_matrix(Y_test,predictions)
print ('Confusion matrix is : ')
print(cm)

Confusion matrix is : 
[[1912  165]
 [  23 2060]]


In [ ]:
print(classification_report(X_test_prediction,Y_test))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      1935
           1       0.99      0.93      0.96      2225

    accuracy                           0.95      4160
   macro avg       0.95      0.96      0.95      4160
weighted avg       0.96      0.95      0.95      4160



In [ ]:
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
conf_matrix

,Predicted:0,Predicted:1
Actual:0,1912,165
Actual:1,23,2060


In [ ]:
plt.figure(figsize = (6,5))
sns.heatmap(conf_matrix, annot=True)

2) Decision Tree:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model1 = DecisionTreeClassifier()
model1.fit(X_train, Y_train)

DecisionTreeClassifier()

In [ ]:
X_train_prediction1 = model1.predict(X_train)
training_data_accuracy1 = accuracy_score(X_train_prediction1,Y_train)
print("Accuracy score of the training data :" , training_data_accuracy1)

Accuracy score of the training data : 1.0


In [ ]:
X_test_prediction1 = model1.predict(X_test)
test_data_accuracy1 = accuracy_score(X_test_prediction1,Y_test)
print("Accuracy score of the test data :" , test_data_accuracy1)

Accuracy score of the test data : 0.9759615384615384


In [ ]:
# Confusion Matrix
prediction1=model1.predict(X_test)
cm1=confusion_matrix(Y_test,prediction1)
print ('Confusion matrix is : ')
print(cm1)

Confusion matrix is : 
[[2034   43]
 [  57 2026]]


In [ ]:
conf_matrix1=pd.DataFrame(data=cm1,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
conf_matrix1

,Predicted:0,Predicted:1
Actual:0,2034,43
Actual:1,57,2026


In [ ]:
plt.figure(figsize = (6,5))
sns.heatmap(conf_matrix1, annot=True, cmap="Blues")

In [ ]:
#model.score(X_test, Y_test)

In [ ]:
print(classification_report(X_test_prediction1,Y_test))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      2091
           1       0.97      0.98      0.98      2069

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160



3) Random Forest:


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model2= RandomForestClassifier(random_state=0)
model2.fit(X_train, Y_train)

RandomForestClassifier(random_state=0)

In [ ]:
X_train_prediction2 = model2.predict(X_train)
training_data_accuracy2 = accuracy_score(X_train_prediction2,Y_train)
print("Accuracy score of the training data :" , training_data_accuracy2)

Accuracy score of the training data : 1.0


In [ ]:
X_test_prediction2 = model2.predict(X_test)
test_data_accuracy2 = accuracy_score(X_test_prediction2,Y_test)
print("Accuracy score of the test data :" , test_data_accuracy2)

Accuracy score of the test data : 0.978125


In [ ]:
# Confusion Matrix
prediction2=model2.predict(X_test)
cm2=confusion_matrix(Y_test,prediction2)
print ('Confusion matrix is : ')
print(cm2)

Confusion matrix is : 
[[2006   71]
 [  20 2063]]


In [ ]:
conf_matrix2=pd.DataFrame(data=cm2,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
conf_matrix2

,Predicted:0,Predicted:1
Actual:0,2006,71
Actual:1,20,2063


In [ ]:
plt.figure(figsize = (6,5))
sns.heatmap(conf_matrix2, annot=True, cmap="Oranges")

In [ ]:
print(classification_report( X_test_prediction2, Y_test))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2026
           1       0.99      0.97      0.98      2134

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160



4) Naive Bayes:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model3= MultinomialNB()
model3.fit(X_train, Y_train)

MultinomialNB()

In [ ]:
X_train_prediction3 = model3.predict(X_train)
training_data_accuracy3 = accuracy_score(X_train_prediction3,Y_train)
print("Accuracy score of the training data :" , training_data_accuracy3)

Accuracy score of the training data : 0.9727764423076923


In [ ]:
X_test_prediction3 = model3.predict(X_test)
test_data_accuracy3 = accuracy_score(X_test_prediction3,Y_test)
print("Accuracy score of the test data :" , test_data_accuracy3)

Accuracy score of the test data : 0.9283653846153846


In [ ]:
# Confusion Matrix
prediction3=model3.predict(X_test)
cm3=confusion_matrix(Y_test,prediction3)
print ('Confusion matrix is : ')
print(cm3)

Confusion matrix is : 
[[2039   38]
 [ 260 1823]]


In [ ]:
conf_matrix3=pd.DataFrame(data=cm3,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
conf_matrix3

,Predicted:0,Predicted:1
Actual:0,2039,38
Actual:1,260,1823


In [ ]:
plt.figure(figsize = (6,5))
sns.heatmap(conf_matrix3, annot=True, cmap="Greys")

In [ ]:
print(classification_report( X_test_prediction3, Y_test))

              precision    recall  f1-score   support

           0       0.98      0.89      0.93      2299
           1       0.88      0.98      0.92      1861

    accuracy                           0.93      4160
   macro avg       0.93      0.93      0.93      4160
weighted avg       0.93      0.93      0.93      4160



#`MODEL ANALYSIS`

In [ ]:
accuracylist=[]

In [ ]:
l=test_data_accuracy
accuracylist.append(l)
d= test_data_accuracy1
accuracylist.append(d)
r=test_data_accuracy2
accuracylist.append(r)
N=test_data_accuracy3
accuracylist.append(N)

In [ ]:
accuracylist

[0.9548076923076924, 0.9759615384615384, 0.978125, 0.9283653846153846]

In [ ]:
mylist2 = ["Logistic Regression","DecisionTree","RandomForest","Naive Bayes"]

In [ ]:
plt.figure(figsize=(15,6))
ax = sns.barplot(x=mylist2, y=accuracylist, palette = "Purples", saturation =1.5)
plt.xlabel("ML Models", fontsize = 20 )
plt.ylabel("% of Accuracy", fontsize = 20)
plt.title("Accuracy of different  Models", fontsize = 20)
plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 13)
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.2%}', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()

#`MAKING A PREDICTIVE SYSTEM`

In [ ]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")


[1]
the news is fake


In [ ]:
X_new = X_test[0]

prediction = model1.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
X_new = X_test[0]

prediction = model2.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
X_new = X_test[0]

prediction = model3.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
print(Y_test[0])

1


In [ ]:
X_new = X_test[55]

prediction = model.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
X_new = X_test[55]

prediction = model1.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
X_new = X_test[55]

prediction = model2.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
X_new = X_test[55]

prediction = model3.predict(X_new)
print(prediction)

if(prediction == 0):
  print("the news is real")
else:
  print("the news is fake")

[1]
the news is fake


In [ ]:
print(Y_test[55]) 

1
